In [2]:
import json
import redis
from pymongo import MongoClient 
# gzip -d artist.json.gz

# 60. KVSの構築
Key-Value-Store (KVS) を用い，アーティスト名（name）から活動場所（area）を検索するためのデータベースを構築せよ．

In [3]:
r = redis.Redis(host='localhost', port=6379, db=0)

In [4]:
with open("artist.json", 'r') as f:
    for line in f.readlines():
        line_json = json.loads(line[:-1])
        if 'area' in line_json.keys():
            r.set(line_json['name'], line_json['area'])

In [5]:
len(r.keys())

309317

# 61. KVSの検索
60で構築したデータベースを用い，特定の（指定された）アーティストの活動場所を取得せよ．

In [6]:
r.get('Al Street')

b'United States'

# 62. KVS内の反復処理
60で構築したデータベースを用い，活動場所が「Japan」となっているアーティスト数を求めよ．

In [7]:
len([key for key in r.scan_iter() if r.get(key) == b'Japan'])

22128

# 63. オブジェクトを値に格納したKVS

KVSを用い，アーティスト名（name）からタグと被タグ数（タグ付けされた回数）のリストを検索するためのデータベースを構築せよ．さらに，ここで構築したデータベースを用い，アーティスト名からタグと被タグ数を検索せよ．

In [8]:
r.flushall()
with open("artist.json", 'r') as f:
    for line in f.readlines()[:1000]:
        line_json = json.loads(line[:-1])
        if 'tags' in line_json.keys():
            r.set(line_json['name'], line_json['tags'])

In [9]:
r.get('Shrek & Fiona')

b"[{'count': 1, 'value': 'bogus artist'}]"

In [10]:
r.flushall()

True

# 64. MongoDBの構築

アーティスト情報（artist.json.gz）をデータベースに登録せよ．さらに，次のフィールドでインデックスを作成せよ: name, aliases.name, tags.value, rating.value

In [19]:
# sudo service mongodb start
# mongoimport --db nlp_artists --collection artists --file artist.json
client = MongoClient('localhost', 27017)
db = client.nlp_artists
artists = db.artists
# artists.drop()

In [20]:
artists.create_index("name")
artists.create_index("aliases.name")
artists.create_index("tags.value")
artists.create_index("rating.value")
artists.index_information() # インデックス確認

{'_id_': {'key': [('_id', 1)], 'ns': 'nlp_artists.artists', 'v': 2},
 'aliases.name_1': {'key': [('aliases.name', 1)],
  'ns': 'nlp_artists.artists',
  'v': 2},
 'name_1': {'key': [('name', 1)], 'ns': 'nlp_artists.artists', 'v': 2},
 'rating.value_1': {'key': [('rating.value', 1)],
  'ns': 'nlp_artists.artists',
  'v': 2},
 'tags.value_1': {'key': [('tags.value', 1)],
  'ns': 'nlp_artists.artists',
  'v': 2}}

# 65. MongoDBの検索
MongoDBのインタラクティブシェルを用いて，"Queen"というアーティストに関する情報を取得せよ．さらに，これと同様の処理を行うプログラムを実装せよ．

In [21]:
# db.artists.find({"name":"Queen"})
[x for x in artists.find({"name" : "Queen"})]

[{'_id': ObjectId('591e43a626513376b372496f'),
  'aliases': [{'name': 'Queen', 'sort_name': 'Queen'}],
  'area': 'Japan',
  'ended': True,
  'gender': 'Female',
  'gid': '420ca290-76c5-41af-999e-564d7c71f1a7',
  'id': 701492,
  'name': 'Queen',
  'sort_name': 'Queen',
  'tags': [{'count': 1, 'value': 'kamen rider w'},
   {'count': 1, 'value': 'related-akb48'}],
  'type': 'Character'},
 {'_id': ObjectId('591e43a926513376b373104f'),
  'aliases': [{'name': '女王', 'sort_name': '女王'}],
  'area': 'United Kingdom',
  'begin': {'date': 27, 'month': 6, 'year': 1970},
  'ended': True,
  'gid': '0383dadf-2a4e-4d10-a46a-e9e041da8eb3',
  'id': 192,
  'name': 'Queen',
  'rating': {'count': 24, 'value': 92},
  'sort_name': 'Queen',
  'tags': [{'count': 2, 'value': 'hard rock'},
   {'count': 1, 'value': '70s'},
   {'count': 1, 'value': 'queen family'},
   {'count': 1, 'value': '90s'},
   {'count': 1, 'value': '80s'},
   {'count': 1, 'value': 'glam rock'},
   {'count': 4, 'value': 'british'},
   {'count

# 66. 検索件数の取得
MongoDBのインタラクティブシェルを用いて，活動場所が「Japan」となっているアーティスト数を求めよ．

In [22]:
# db.artists.count({"area":"Japan"})
artists.count({"aliases.name":""})

0

# 67. 複数のドキュメントの取得
特定の（指定した）別名を持つアーティストを検索せよ．

In [23]:
artists.find_one({"aliases.name" : "Marilyn Manson"})

{'_id': ObjectId('591e43a526513376b3721b9f'),
 'aliases': [{'name': 'Marilyn Manson', 'sort_name': 'Marilyn Manson'},
  {'name': 'The Spooky Kids', 'sort_name': 'The Spooky Kids'},
  {'name': 'Marilyn Manson and the Spooky Kids',
   'sort_name': 'Marilyn Manson and the Spooky Kids'}],
 'area': 'United States',
 'begin': {'year': 1989},
 'end': {'year': 1992},
 'ended': True,
 'gid': 'af3dd900-679f-48ef-909a-0da55786b078',
 'id': 73830,
 'name': 'Marilyn Manson & the Spooky Kids',
 'sort_name': 'Manson, Marilyn & the Spooky Kids',
 'type': 'Group'}

# 68. ソート
"dance"というタグを付与されたアーティストの中でレーティングの投票数が多いアーティスト・トップ10を求めよ．

In [24]:
dance = [x for x in artists.find({"tags.value":"dance"}) if 'rating' in x.keys()]
[(x['name'],x['rating']) for x in sorted(dance, key=lambda x:x["rating"]["count"], reverse=True)[:10]]

[('Madonna', {'count': 26, 'value': 88}),
 ('Björk', {'count': 23, 'value': 84}),
 ('The Prodigy', {'count': 23, 'value': 90}),
 ('Rihanna', {'count': 15, 'value': 68}),
 ('Britney Spears', {'count': 13, 'value': 83}),
 ('Maroon 5', {'count': 11, 'value': 60}),
 ('Adam Lambert', {'count': 7, 'value': 100}),
 ('Fatboy Slim', {'count': 7, 'value': 77}),
 ('Basement Jaxx', {'count': 6, 'value': 83}),
 ('Cornershop', {'count': 5, 'value': 68})]

# 69. Webアプリケーションの作成
ユーザから入力された検索条件に合致するアーティストの情報を表示するWebアプリケーションを作成せよ．アーティスト名，アーティストの別名，タグ等で検索条件を指定し，アーティスト情報のリストをレーティングの高い順などで整列して表示せよ．

webアプリの起動

`$ python app.py`

起動後ローカルホストにアクセスして利用
フレームワークbottleで開発